# Detection de motifs


## BruteForce & Median String

In [1]:
import random

nucl = ["A","T","G","C"]

k=4 #taille de motif
v=1 #nb de positions variable dans le motif
t=3 #nb de sequences
n=10 #longuer des sequence

def generateRandomSequences(n, t):
    return [generateRandomSequence(n) for k in range(0,t)]

def generateRandomSequence(n):
    return random.choices(nucl,weights =[1,1,1,1],k=n)

def implantMotifVar(motif, sequences, v):
    res = []
    for s in sequences:
        m = motif
        for i in range(v):       
            pos = int(random.uniform(0,len(motif)))
            substituant = int(random.uniform(0,len(nucl)))
            m[pos] = nucl[substituant]
        
        pos = int(random.uniform(0,len(s)))
        res.append(s[:pos]+m+s[pos:])
    return res

sequences = generateRandomSequences(n, t)
motif = generateRandomSequence(k)
sequences = implantMotifVar(motif, sequences, v)



In [2]:
k=8  #taille de motif
v=2  #nb de positions variable dans le motif
t=10 #nb de sequences
n=10 #longuer des sequence

In [3]:
sequences = generateRandomSequences(n, t)
motif = generateRandomSequence(k)
sequences = implantMotifVar(motif, sequences, v)

for s in sequences:
    s = ''.join(s) 
    print(s)


GAGTTACTAAAGGTTTGG
TCACAAGCTCTGCTGGTT
TACTAGAATGCTCTGAGT
GTTATCCTGTTCTGGCGT
GGAGCGCTGTTCGGGCCC
TCCCGTGCGGTGAGAGTG
TGACCCTTCGGCGTGACA
GTGCGCCCCTTTGGGCTC
CGTACTCCCCCTCCGGCC
GGGGCCGCCGGGGTATAG


In [4]:
import numpy as np

nucl = ["A","T","G","C"]

k = 8  # taille de motif
n = 68 #longuer des sequence
t = 5  #nb de sequences
q = 4
s=(25,20,2,55,59)

seqs =[
'CCTGATAGACGCTATCTGGCTATCCAGGTACTTAGGTCCTCTGTGCGAATCTATGCGTTTCCAACCAT',
'AGTACTGGTGTACATTTGATCCATACGTACACCGGCAACCTGAAACAAACGCTCAGAACCAGAAGTGC',
'AAACGTTAGTGCACCCTCTTTCTTCGTGGCTCTGGCCAACGAGGGCTGATGTATAAGACGAAAATTTT',
'AGCCTCCGATGTAAGTCATAGCTGTAACTATTACCTGCCACCCCTATTACATCTTACGTCCATATACA',
'CTGTTATACAACGCGTCATGGCGGGGTATGCGTTTTGGTCGTCGTACGCTCGATCGTTACCGTACGGC'
]



#extract sequences from seqs
def extractSeqs(s,k,seqs):
    motifs = []   
    for i in range(len(s)):
        motifs.append(seqs[i][s[i]:s[i]+k])
   
    return motifs


#construct Profiles
def profile(motifs, k, t, q):
    PWM = np.zeros((q, k))
    for i in range(k):
        for m in motifs:
            for j in range(q):
                if m[i]==nucl[j]:
                    PWM[j][i]+=1
    return PWM


def score(s,seqs,k,t,q):
    sc = 0
    pwm = profile(extractSeqs(s,k, seqs), k, t, q)
    for i in range(k):
        sc += max([pwm[j][i] for j in range(q)])
    motif = getBestMotifFromMatrix(pwm, k, q)
    return (sc, motif)

def getBestMotifFromMatrix(pwm, k, q):
    motif = ''
    for i in range(k):
        values = [pwm[j][i] for j in range(q)]
        maxVal = max(values)
        for j in range(q):
            if values[j] == maxVal:
                motif += nucl[j]
    return motif
            
sc = score(s, seqs, k, t ,q)
#teste, sc doit etre egalle à 30
assert(sc[0]==30)
print ('\n',sc[0])




 30.0


In [5]:
from itertools import product


#enumeration all initial positions
def initialPosition(k, n, t):
	initPos = []; allInitPos = [];
	for i in range(0, n-k+1):
		initPos.append(i)
        
	for subset in product(initPos, repeat=t):
		allInitPos.append(subset)
	return allInitPos

#Pour tester la generation de positions
#print (initialPosition(k, n, t))

In [6]:
def bruteForceMotifSearch(sequences, t, n, k, q):
    bestScore = 0; bestMotif=''
    initialPositions = initialPosition(k,n,t)
    for s in initialPositions:
        currScore = score(s, sequences, k, t, q)
        if currScore[0] > bestScore:
            bestScore = currScore[0]
            bestMotif = currScore[1]
    return bestMotif, bestScore

#test
k=4; v=1; t=3; n=10 ;q=4

sequences = generateRandomSequences(n, t)
motif = generateRandomSequence(k)
sequences = implantMotifVar(motif, sequences, v)

bestMotif, bestScore = bruteForceMotifSearch(sequences, t, n, k, q)
print (bestMotif, bestScore)

GGAC 10.0


In [7]:
k=7; v=1; t=5; n=20; q= 4

sequences = generateRandomSequences(n, t)
motif = generateRandomSequence(k)
sequences = implantMotifVar(motif, sequences, v)
bestMotif, bestScore = bruteForceMotifSearch(sequences, t, n, k, q)

print (bestMotif, bestScore)

GTTAGGC 26.0


In [8]:

nucl = ["A","T","G","C"]

k = 8  # taille de motif
n = 68 #longuer des sequence
t = 5  #nb de sequences
q = 4
s=(25,20,2,55,59)

seqs =[
'CCTGATAGACGCTATCTGGCTATCCAGGTACTTAGGTCCTCTGTGCGAATCTATGCGTTTCCAACCAT',
'AGTACTGGTGTACATTTGATCCATACGTACACCGGCAACCTGAAACAAACGCTCAGAACCAGAAGTGC',
'AAACGTTAGTGCACCCTCTTTCTTCGTGGCTCTGGCCAACGAGGGCTGATGTATAAGACGAAAATTTT',
'AGCCTCCGATGTAAGTCATAGCTGTAACTATTACCTGCCACCCCTATTACATCTTACGTCCATATACA',
'CTGTTATACAACGCGTCATGGCGGGGTATGCGTTTTGGTCGTCGTACGCTCGATCGTTACCGTACGGC'
]


In [9]:
from itertools import product
allkmers = [''.join(c) for c in product('ATGC', repeat=k)]

In [10]:
# TME 4 
import re, operator
m = 4

def isSimple(motif, m):
    for c in motif:
        if len([s.start() for s in re.finditer(c, motif)]) >= m:
               return True
    return False
               
def isSimplePair(motif):
    for i in range(len(motif)-2):    
        if motif[i+1:].find(motif[i:i+2]) != -1:
            return True
    return False
    
def removeLowComplexe(motifs, minrep):
    validMotif = []
    for m in motifs:
        if not isSimple(m, minrep):
               validMotif.append(m)
    return validMotif

def removeLowComplexePair(motifs):
    validMotif = []
    for m in motifs:
        if not isSimplePair(m):
            validMotif.append(m)
    return validMotif


def getKmersFromSeq(s, k):
    s = ''.join(s)
    res = [s[i:i+k] for i in range(len(s))]
    res = [s for s in res if len(s)==k]
    return res

In [11]:
def getComplexKmers(validkmers, m): 
    return removeLowComplexePair(removeLowComplexe(allkmers, m))

allkmers = getComplexKmers(allkmers,m)
print(allkmers)

['AATAGTTG', 'AATAGTTC', 'AATAGTGG', 'AATAGTGC', 'AATAGTCT', 'AATAGTCG', 'AATAGTCC', 'AATAGGTT', 'AATAGGTG', 'AATAGGTC', 'AATAGGCT', 'AATAGGCG', 'AATAGGCC', 'AATAGCTT', 'AATAGCTG', 'AATAGCTC', 'AATAGCGT', 'AATAGCGG', 'AATAGCCT', 'AATAGCCG', 'AATACTTG', 'AATACTTC', 'AATACTGT', 'AATACTGG', 'AATACTGC', 'AATACTCG', 'AATACTCC', 'AATACGTT', 'AATACGTG', 'AATACGTC', 'AATACGGT', 'AATACGGC', 'AATACGCT', 'AATACGCC', 'AATACCTT', 'AATACCTG', 'AATACCTC', 'AATACCGT', 'AATACCGG', 'AATACCGC', 'AATTAGTG', 'AATTAGTC', 'AATTAGGT', 'AATTAGGC', 'AATTAGCT', 'AATTAGCG', 'AATTAGCC', 'AATTACTG', 'AATTACTC', 'AATTACGT', 'AATTACGG', 'AATTACGC', 'AATTACCT', 'AATTACCG', 'AATTGAGT', 'AATTGAGG', 'AATTGAGC', 'AATTGACT', 'AATTGACG', 'AATTGACC', 'AATTGTAG', 'AATTGTAC', 'AATTGTCA', 'AATTGTCG', 'AATTGTCC', 'AATTGGAG', 'AATTGGAC', 'AATTGGTA', 'AATTGGTC', 'AATTGGCA', 'AATTGGCT', 'AATTGGCG', 'AATTGGCC', 'AATTGCAG', 'AATTGCAC', 'AATTGCTA', 'AATTGCTC', 'AATTGCGA', 'AATTGCGT', 'AATTGCGG', 'AATTGCCA', 'AATTGCCT', 'AATTGCCG', 'AA

In [20]:
m = 4
def hamdist(v, x):
    x=''.join(x)
    if len(v) != len(x):
        return len(v)
    res = 0
    for i in range(len(v)):
        if v[i]!=x[i]:
            res+=1
    return res


def minHamdistKmer(v, kmers):
    minHamDist = len(v)
    minKmer = ''
    for k in kmers:
        currHamDist = hamdist(v, k)
        if currHamDist < minHamDist:
            minHamDist = currHamDist
            minKmer = k
    return k, minHamDist

def totalDistance(motif, sequences, k):
    td = 0
    for s in sequences:
        kmers = getKmersFromSeq(s, k)
        kmer, hamDist = minHamdistKmer(motif, kmers)
        td += hamDist
    return td


def MedianStringSearch(allkmers, sequences, t, n, k):
    bestDistance = 1000
    bestMotif = ""
    motifDist = dict()
    for v in allkmers:
        td = totalDistance(v, sequences, k)
        if v not in motifDist:
            motifDist[v] = td
        if td < bestDistance:
            bestDistance = td
            bestMotif = v
            
    return bestMotif, bestDistance, motifDist

td = totalDistance("ACGTACGT", sequences, k)
print (td)

bestMotif, bestDistance, motifDist = MedianStringSearch(allkmers , sequences, t, n, k)
print (bestMotif, bestDistance)

40
AGTGACG 8


In [21]:
import time
from itertools import product

k=7; v=1; t=5; n=20; q = 4;
sequences = [''.join(s) for s in generateRandomSequences(n, t)]

# Median String
start_time = time.time()
allkmers = [''.join(c) for c in product('ATGC', repeat=k)]
allkmers = getComplexKmers(allkmers,m)
a,b,c = MedianStringSearch(allkmers, sequences, t, n, k)
print(a,b)
print("temps d'exécution de Median String Search: %.4s seconds" % (time.time() - start_time))

# Brute Force
start_time = time.time()
a,b = bruteForceMotifSearch(sequences, t, n, k, q)
print(a,b)
print("temps d'exécution de Brute Force Search: %.4s seconds" % (time.time() - start_time))

CACCTGT 9
temps d'exécution de Median String Search: 0.97 seconds


KeyboardInterrupt: 